# Projects markdown generator for academicpages

Takes a TSV of projects with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `projects.py`. Run either from the `markdown_generator` folder after replacing `projects.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: date, title, venue, excerpt, with a header at the top.

- `excerpt` and `paper_url` can be blank, but the others must have values.
- `date` must be formatted as YYYY-MM-DD.

In [19]:
!cat projects.tsv

date	title	venue	excerpt
2022-07-25	Artificial Intelligence in Modern Board Games	Ohio Wesleyan University	Developed an AI opponent for the card game Lost Cities using object oriented principles in Java.
2022-07-31	AIParkinScan	MITxSureStart FutureMakers Create-a-Thon Program	Co-developed AIParkinScan software for Parkinson's diagnosis using audio and image data. Utilized neural networks, spectrograms, and Random Forest algorithm.
2022-11-30	Connect 4 AI	Ohio Wesleyan University	Co-developed an AI opponent for Connect 4 using alpha-beta pruning algorithm.
2023-04-30	Artificial Intelligence Opinion Survey	Ohio Wesleyan University	Designed, fielded, and investigated a survey research of 1000 employees to analyze AI's impact on work efficiency and attitudes. Found AI improved efficiency across industries.


## Import pandas

We are using the very handy pandas library for dataframes.

In [20]:
import os
import datetime
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [21]:
projects = pd.read_csv("projects.tsv", sep="\t", header=0)
projects


,date,title,venue,excerpt
0,2022-07-25,Artificial Intelligence in Modern Board Games,Ohio Wesleyan University,Developed an AI opponent for the card game Los...
1,2022-07-31,AIParkinScan,MITxSureStart FutureMakers Create-a-Thon Program,Co-developed AIParkinScan software for Parkins...
2,2022-11-30,Connect 4 AI,Ohio Wesleyan University,Co-developed an AI opponent for Connect 4 usin...
3,2023-04-30,Artificial Intelligence Opinion Survey,Ohio Wesleyan University,"Designed, fielded, and investigated a survey r..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [22]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
}

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c, c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [25]:
for row, item in projects.iterrows():

    # create a string url slug from the title
    url_slug = str(item.title).replace(" ", "-").lower()
    url_slug = url_slug.replace("&", "and")
    url_slug = url_slug.replace("/", "-")
    url_slug = url_slug.replace("(", "")
    url_slug = url_slug.replace(")", "")
    url_slug = url_slug.replace(":", "")
    url_slug = url_slug.replace(";", "")
    url_slug = url_slug.replace("?", "")
    url_slug = url_slug.replace("!", "")
    url_slug = url_slug.replace(",", "")
    url_slug = url_slug.replace(".", "")
    url_slug = url_slug.replace("=", "")
    url_slug = url_slug.replace("+", "")

    date_no_day = item.date[:8]
    html_filename = str(item.date[:7]) + "-" + url_slug
    md_filename = html_filename + ".md"
    year = item.date[:4]

    # YAML variables

    md = "---"

    md += "\ndate: " + str(item.date)

    md += "\ntitle: \"" + item.title + '"\n'

    md += """collection: projects"""

    md += """\npermalink: /projects/""" + html_filename

    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: \"" + html_escape(item.excerpt) + "\""

    md += "\nvenue: \"" + html_escape(item.venue) + "\""

    md += "\nlast_modified_at: " + \
        str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    md += "\n---"

    # Markdown description for individual page

    md += "\n\n### Skills"
    md += "\n\n- skill 1"
    md += "\n- skill 2"
    md += "\n- skill 3"
    md += "\n\n## Links"
    md += "\n\n- [GitHub Repository](https://github.com/Aadarsha2002/<repository_name>)"

    md_filename = os.path.basename(md_filename)

    with open("../_projects/" + md_filename, 'w') as f:
        f.write(md)


These files are in the projects directory, one directory below where we're working from.

In [9]:
!ls ../_projects/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md


In [10]:
!cat ../_projects/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).
